# Full Pipeline

## Imports

In [1]:
import os
from PIL import Image
from sahi.model import MmdetDetectionModel
from sahi.predict import get_prediction, get_sliced_prediction, predict
from sahi.utils.cv import visualize_object_predictions
import numpy as np

/vol/bitbucket/jrb21/anaconda3/envs/sodb/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Google Earth Image resize

In [2]:
image_sizes = set()

for directory in os.listdir('warwickshire_images'):
    print('dir:',directory)
    for file in os.listdir(os.path.join('warwickshire_images', directory)):
        print('----',file)
        image = Image.open(os.path.join('warwickshire_images', directory, file))
        image_sizes.add(image.size) # Output: (400, 267)

print(image_sizes)

dir: 2017
---- 7153_4-8-2017.jpg
---- 84047_4-8-2017.jpg
---- 73741_4-8-2017.jpg
---- 84042_4-8-2017.jpg
---- 57893_4-8-2017.jpg
---- 99324_4-8-2017.jpg
---- 48615_4-8-2017.jpg
---- 49698_4-8-2017.jpg
---- 58335_4-8-2017.jpg
---- 84027_4-8-2017.jpg
---- 48711_4-8-2017.jpg
---- 84026_4-8-2017.jpg
---- 38717_4-8-2017.jpg
---- 6142_4-8-2017.jpg
---- 89130_4-8-2017.jpg
---- 89134_4-8-2017.jpg
dir: 2019
---- 7153_5-14-2019.jpg
---- 89134_5-14-2019.jpg
---- 6142_5-14-2019.jpg
---- 84042_5-12-2019.jpg
---- 84047_5-12-2019.jpg
---- 38717_5-14-2019.jpg
---- 84047_5-14-2019.jpg
---- 84026_5-14-2019.jpg
---- 84042_5-14-2019.jpg
---- 89130_5-12-2019.jpg
---- 49698_5-14-2019.jpg
---- 84027_5-14-2019.jpg
---- 57893_5-14-2019.jpg
---- 84027_5-12-2019.jpg
---- 73741_5-14-2019.jpg
---- 89130_5-14-2019.jpg
dir: 2021
---- 99324_6-15-2021.jpg
---- 58322_6-16-2021.jpg
---- 38708_6-15-2021.jpg
---- 8625_6-16-2021.jpg
---- 7153_4-4-2021.jpg
---- 84047_4-4-2021.jpg
---- 73851_6-16-2021.jpg
---- 99324_4-22-202

IsADirectoryError: [Errno 21] Is a directory: 'warwickshire_images/2020/.ipynb_checkpoints'

### Make the directory for the resized images

In [19]:
os.mkdir('warwickshire_images_resized')
for dirname in os.listdir('warwickshire_images'):
    os.mkdir(os.path.join('warwickshire_images_resized', dirname))

### Resize the image

On all the images we see that the 60m scale is 310 pixels long. 

We want a GSD of 0.3m therefore we want 60m = 60/0.3 = 200 pixels.

Therefore we multiply the widths and heights by 200/310

In [3]:
width, height = image.size
new_width, new_height = width * 200/310, height * 200/310
new_width, new_height

(3096.7741935483873, 1927.0967741935483)

In [6]:
for directory in os.listdir('warwickshire_images'):
    # Directorie are the years
    for file in os.listdir(os.path.join('warwickshire_images', directory)):
        if file.split('.')[-1] == 'jpg':
            # The image files
            image = Image.open(os.path.join('warwickshire_images', directory, file))
            image.thumbnail((new_width, new_height))
            image.save(os.path.join('warwickshire_images_resized', directory, file))

In [11]:
image.size

(3096, 1927)

## Inference

### Test function: get_sliced_prediction

In [2]:
detection_model = MmdetDetectionModel(
    model_path= "models-to-test/latest.pth",
    config_path= "mmdet_configs/xview_tood/tood_crop_300_500_cls_cars_trucks_1e-3_new_pipe.py",
    device='cuda' # or 'cpu'
)

load checkpoint from local path: models-to-test/latest.pth


In [3]:
test_img_path = "warwickshire_images_resized/2020/77206_4-21-2020.jpg"
width, height = (400,400)

sliced_pred_result = get_sliced_prediction(
    test_img_path,
    detection_model, 
    slice_width=width, 
    slice_height=height
)

Performing prediction on 60 number of slices.


In [21]:
sliced_pred_result.object_prediction_list[0].bbox.__dict__.items()

dict_items([('minx', 1266), ('miny', 975), ('maxx', 1285), ('maxy', 990), ('shift_x', 0), ('shift_y', 0)])

In [13]:
min([item.score.value for item in sliced_pred_result.object_prediction_list])

0.3079700469970703

It appears that the function get_sliced_prediction has a confidence threshold of 0.3.

In [16]:
viz_path = "test_result"
test_img_name = "test"


visualize_object_predictions(
    image=np.ascontiguousarray(sliced_pred_result.image),
    object_prediction_list=sliced_pred_result.object_prediction_list,
    rect_th=1,
    text_size=0.3,
    text_th=1,
    color=(0, 0, 0),
    output_dir=viz_path,
    file_name=test_img_name + "_sliced_pred_result",
    export_format="png",
)

{'image': array([[[ 97, 100,  93],
         [100, 103,  96],
         [ 91,  94,  85],
         ...,
         [ 25,  44,  22],
         [ 16,  35,  13],
         [ 34,  53,  31]],
 
        [[ 98, 101,  92],
         [ 87,  90,  81],
         [ 82,  85,  76],
         ...,
         [ 24,  43,  21],
         [ 13,  32,  10],
         [ 27,  46,  24]],
 
        [[ 76,  79,  70],
         [ 53,  56,  47],
         [ 53,  56,  47],
         ...,
         [ 31,  49,  27],
         [ 14,  32,  10],
         [ 18,  36,  14]],
 
        ...,
 
        [[136, 125, 103],
         [137, 126, 104],
         [145, 133, 111],
         ...,
         [ 34,  53,  31],
         [ 64,  84,  59],
         [ 59,  79,  54]],
 
        [[140, 129, 107],
         [141, 130, 108],
         [148, 136, 114],
         ...,
         [ 42,  61,  39],
         [ 54,  74,  49],
         [ 44,  64,  39]],
 
        [[142, 131, 109],
         [142, 131, 109],
         [148, 136, 114],
         ...,
         [ 55,  74,

### Batch Prediction